In [2]:
import pandas as pd
import openpyxl
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import normalize

In [3]:
def prepare_season_data(title):
    
    global season, season_dropped, season_filled, season_scores
    
    season = pd.read_csv(f"C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\siege stats - {title}.csv")
    season = season.drop(["Game #", "Quick Notes"], axis = 1)
    season = season.drop(season.tail(1).index)
    

    # standardizing "Outcome" observations
    outcomes = list(season["Outcome"])
    outcomes_new = []
    for i in range(len(outcomes)):
        outcomes_new.append(outcomes[i].lower())
    season["Outcome"] = outcomes_new
    
    # dropping ALL nan values
    season_dropped = season.dropna()
    
    # filling all nan values with 0
    season_filled = season.fillna(0)
    
    # preparing data for mean score and standardizing scores for wins and losses
        # siege adds 2000 pts for a round win, so we take those back out here
    # this section might be its own function starting y8s4
    season_scores = season.dropna(subset = ["Josh Score"])
    season_scores.fillna(0)
    season_scores["Adjusted Score"] = season_scores.loc[season_scores["Outcome"] == "win", "Josh Score"] - 2000
    season_scores["Adjusted Score"] = season_scores["Adjusted Score"].fillna(season_scores.loc[season_scores["Outcome"] == "loss", "Josh Score"])
    
    return season, season_dropped, season_filled, season_scores

def prepare_lifetime_data():
    
    global lifetime, lifetime_dropped, lifetime_filled, lifetime_scores
    
    lifetime = pd.read_csv(f"C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\siege stats - everything.csv")
    lifetime = lifetime.drop("Game #", axis = 1)

    # standardizing "Outcome" observations
    outcomes = list(lifetime["Outcome"])
    outcomes_new = []
    for i in range(len(outcomes)):
        outcomes_new.append(outcomes[i].lower())
    lifetime["Outcome"] = outcomes_new
    
    # dropping ALL nan values
    lifetime_dropped = lifetime.dropna()
    
    # filling all nan values with 0
    lifetime_filled = lifetime.fillna(0)
    
    # preparing data for mean score and standardizing scores for wins and losses
    lifetime_scores = lifetime.dropna(subset = ["Josh Score"])
    lifetime_scores.fillna(0)
    lifetime_scores["Adjusted Score"] = lifetime_scores.loc[lifetime_scores["Outcome"] == "win", "Josh Score"] - 2000
    lifetime_scores["Adjusted Score"] = lifetime_scores["Adjusted Score"].fillna(lifetime_scores.loc[lifetime_scores["Outcome"] == "loss", "Josh Score"])
    
    return lifetime, lifetime_dropped, lifetime_filled, lifetime_scores

In [4]:

prepare_season_data("Y8S3 - Heavy Mettle") #update for current season
prepare_lifetime_data()

C:\Users\aethe\AppData\Local\Temp\ipykernel_14036\866001529.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_scores["Adjusted Score"] = season_scores.loc[season_scores["Outcome"] == "win", "Josh Score"] - 2000
C:\Users\aethe\AppData\Local\Temp\ipykernel_14036\866001529.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_scores["Adjusted Score"] = season_scores["Adjusted Score"].fillna(season_scores.loc[season_scores["Outcome"] == "loss", "Josh Score"])
C:\Users\aethe\AppData\Local\Temp\ip

(    Season Warm-Up         Map  Squad Size  DEF Wins  DEF Losses  ATK Wins  \
 0     Y7S4     NaN   Coastline         NaN       NaN         NaN       NaN   
 1     Y7S4     NaN  Theme Park         NaN       NaN         NaN       NaN   
 2     Y7S4     NaN      Chalet         NaN       NaN         NaN       NaN   
 3     Y7S4     NaN       Villa         NaN       NaN         NaN       NaN   
 4     Y7S4     NaN  Club House         NaN       NaN         NaN       NaN   
 ..     ...     ...         ...         ...       ...         ...       ...   
 567   Y8S3      th   Coastline         2.0       2.0         2.0       3.0   
 568   Y8S3      th  Skyscraper         2.0       NaN         NaN       NaN   
 569   Y8S3    none  Club House         4.0       1.0         2.0       1.0   
 570   Y8S3    none  Club House         3.0       2.0         0.0       2.0   
 571   Y8S3    none      Chalet         3.0       2.0         1.0       2.0   
 
      ATK Losses Outcome  Josh Score  Kills  Death

In [5]:
# lists I need

levels = list(season["Map"].unique())
levels = sorted(levels)
# have to do something about this for y8s4


sizes = list(season["Squad Size"].unique())
sizes = sorted(sizes)

In [6]:
# getting some overarching statistics

season_kills = season["Kills"].sum()
season_assists = season["Assists"].sum() * (1/3)
season_deaths = season["Deaths"].sum()
season_wins = len(season[(season["Outcome"] == "win")])
season_losses = len(season[(season["Outcome"] == "loss")])

lifetime_kills = lifetime["Kills"].sum()
lifetime_assists = lifetime["Assists"].sum() * 1/3
lifetime_deaths = lifetime["Deaths"].sum()
lifetime_wins = len(lifetime[(lifetime["Outcome"] == "win")])
lifetime_losses = len(lifetime[(lifetime["Outcome"] == "loss")])

avg_season_kd = season_kills/season_deaths
avg_season_kda = (season_kills + season_assists)/season_deaths
avg_season_wl = season_wins/season_losses
avg_season_score = season_score = season_scores["Adjusted Score"].sum()/len(season_scores)

avg_lifetime_kd = lifetime_kills/lifetime_deaths
avg_lifetime_kda = (lifetime_kills + lifetime_assists)/lifetime_deaths
avg_lifetime_wl = lifetime_wins/lifetime_losses
avg_lifetime_score = lifetime_scores["Adjusted Score"].sum()/len(lifetime_scores)


In [7]:
#  setting up individual player score data

josh_scores = season.dropna(subset = ["Josh Score"])
josh_scores.fillna(0)
josh_scores["Adjusted Score"] = josh_scores.loc[josh_scores["Outcome"] == "win", "Josh Score"] - 2000
josh_scores["Adjusted Score"] = josh_scores["Adjusted Score"].fillna(josh_scores.loc[josh_scores["Outcome"] == "loss", "Josh Score"])

josh_total = josh_scores["Adjusted Score"].sum()
josh_count_season = len(josh_scores)
josh_all_levels_avg = josh_total/josh_count_season


asa_scores = season.dropna(subset = ["Asa Score"])
asa_scores.fillna(0)
asa_scores["Adjusted Score"] = asa_scores.loc[asa_scores["Outcome"] == "win", "Asa Score"] - 2000
asa_scores["Adjusted Score"] = asa_scores["Adjusted Score"].fillna(asa_scores.loc[asa_scores["Outcome"] == "loss", "Asa Score"])

asa_total = asa_scores["Adjusted Score"].sum()
asa_count_season = len(asa_scores)
asa_all_levels_avg = asa_total/asa_count_season


ed_scores = season.dropna(subset = ["Ed Score"])
ed_scores.fillna(0)
ed_scores["Adjusted Score"] = ed_scores.loc[ed_scores["Outcome"] == "win", "Ed Score"] - 2000
ed_scores["Adjusted Score"] = ed_scores["Adjusted Score"].fillna(ed_scores.loc[ed_scores["Outcome"] == "loss", "Ed Score"])

ed_total = ed_scores["Adjusted Score"].sum()
ed_count_season = len(ed_scores)
ed_all_levels_avg = ed_total/ed_count_season


luke_scores = season.dropna(subset = ["Luke Score"])
luke_scores.fillna(0)
luke_scores["Adjusted Score"] = luke_scores.loc[luke_scores["Outcome"] == "win", "Luke Score"] - 2000
luke_scores["Adjusted Score"] = luke_scores["Adjusted Score"].fillna(luke_scores.loc[luke_scores["Outcome"] == "loss", "Luke Score"])

luke_total = luke_scores["Adjusted Score"].sum()
luke_count_season = len(luke_scores)
luke_all_levels_avg = luke_total/luke_count_season

C:\Users\aethe\AppData\Local\Temp\ipykernel_14036\3449436871.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  josh_scores["Adjusted Score"] = josh_scores.loc[josh_scores["Outcome"] == "win", "Josh Score"] - 2000
C:\Users\aethe\AppData\Local\Temp\ipykernel_14036\3449436871.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  josh_scores["Adjusted Score"] = josh_scores["Adjusted Score"].fillna(josh_scores.loc[josh_scores["Outcome"] == "loss", "Josh Score"])
C:\Users\aethe\AppData\Local\Temp\ipykernel_14036\

In [8]:
# anonymous player score data

# p1_scores = season.dropna(subset = ["P1 Score"])
# p1_scores.fillna(0)
# p1_scores["Adjusted Score"] = p1_scores.loc[p1_scores["Outcome"] == "win", "P1 Score"] - 2000
# p1_scores["Adjusted Score"] = p1_scores["Adjusted Score"].fillna(p1_scores.loc[p1_scores["Outcome"] == "loss", "P1 Score"])

# p1_total = p1_scores["Adjusted Score"].sum()
# p1_count_season = len(p1_scores)
# p1_all_levels_avg = p1_total/p1_count_season


# p2_scores = season.dropna(subset = ["P2 Score"])
# p2_scores.fillna(0)
# p2_scores["Adjusted Score"] = p2_scores.loc[p2_scores["Outcome"] == "win", "P2 Score"] - 2000
# p2_scores["Adjusted Score"] = p2_scores["Adjusted Score"].fillna(p2_scores.loc[p2_scores["Outcome"] == "loss", "P2 Score"])

# p2_total = p2_scores["Adjusted Score"].sum()
# p2_count_season = len(p2_scores)
# p2_all_levels_avg = p2_total/p2_count_season


# p3_scores = season.dropna(subset = ["P3 Score"])
# p3_scores.fillna(0)
# p3_scores["Adjusted Score"] = p3_scores.loc[p3_scores["Outcome"] == "win", "P3 Score"] - 2000
# p3_scores["Adjusted Score"] = p3_scores["Adjusted Score"].fillna(p3_scores.loc[p3_scores["Outcome"] == "loss", "P3 Score"])

# p3_total = p1_scores["Adjusted Score"].sum()
# p3_count_season = len(p3_scores)
# p3_all_levels_avg = p3_total/p3_count_season


# p4_scores = season.dropna(subset = ["P4 Score"])
# p4_scores.fillna(0)
# p4_scores["Adjusted Score"] = p1_scores.loc[p1_scores["Outcome"] == "win", "P4 Score"] - 2000
# p4_scores["Adjusted Score"] = p4_scores["Adjusted Score"].fillna(p4_scores.loc[p1_scores["Outcome"] == "loss", "P4 Score"])

# p4_total = p4_scores["Adjusted Score"].sum()
# p4_count_season = len(p4_scores)
# p4_all_levels_avg = p4_total/p4_count_season

In [9]:
season_dummies = season.drop(["Session",
                              "Map",
                              "RP",
                              "ΔRP",
                              "Outcome",
                              "Jack Score",
                              "Cousin Score",
                              "Aidan Score",
                              "Biggs Score",
                              "Conell Score"],
                              axis = 1)

season_dummies = pd.get_dummies(season_dummies[["Warm-Up"]])
season_dummies["Kills"] = season["Kills"]
season_dummies["Deaths"] = season["Deaths"]
season_dummies["Assists"] = season["Assists"]
season_dummies["Score"] = josh_scores["Adjusted Score"]
season_dummies["Outcome"] = season["Outcome"]
season_dummies = season_dummies.dropna()

season_dummies["Score"] = (season_dummies["Score"] - season_dummies["Score"].min()) / (season_dummies["Score"].max() - season_dummies["Score"].min())

In [10]:
def win_loss_level(level_name):
    
    global season_level_wl, lifetime_level_wl
    
    season_filtered = season[season["Map"] == level_name]
    lifetime_filtered = lifetime[lifetime["Map"] == level_name]
    
    season_wins = len(season_filtered[(season_filtered["Outcome"] == "win")])
    season_losses = len(season_filtered[(season_filtered["Outcome"] == "loss")])
    lifetime_wins = len(lifetime_filtered[(lifetime_filtered["Outcome"] == "win")])
    lifetime_losses = len(lifetime_filtered[(lifetime_filtered["Outcome"] == "loss")])
    
    if season_losses == 0:
        season_level_wl = season_wins
    else:
        season_level_wl = season_wins/season_losses
    
    if lifetime_losses == 0:
        lifetime_level_wl = lifetime_wins
    else:
        lifetime_level_wl = lifetime_wins/lifetime_losses
    
    return season_level_wl, lifetime_level_wl

def win_loss_squad(squad_size):
    
    global season_squad_wl, lifetime_squad_wl
    
    season_filtered = season[season["Squad Size"] == squad_size]
    lifetime_filtered = lifetime[lifetime["Squad Size"] == squad_size]
    
    season_wins = len(season_filtered[(season_filtered["Outcome"] == "win")])
    season_losses = len(season_filtered[(season_filtered["Outcome"] == "loss")])
    lifetime_wins = len(lifetime_filtered[(lifetime_filtered["Outcome"] == "win")])
    lifetime_losses = len(lifetime_filtered[(lifetime_filtered["Outcome"] == "loss")])
    
    if season_losses == 0:
        season_squad_wl = season_wins
    else:
        season_squad_wl = season_wins/season_losses
    
    if lifetime_losses == 0:
        lifetime_squad_wl = lifetime_wins
    else:
        lifetime_squad_wl = lifetime_wins/lifetime_losses
        
    return season_squad_wl, lifetime_squad_wl

In [11]:
def kill_death_level(level_name):
    
    global season_level_kd, season_level_kda, lifetime_level_kd, lifetime_level_kda
    
    season_filtered = season[season["Map"] == level_name]
    lifetime_filtered = lifetime[lifetime["Map"] == level_name]
    
    season_kills = season_filtered["Kills"].sum()
    season_assists = season_filtered["Assists"].sum() * (1/3)
    season_deaths = season_filtered["Deaths"].sum()
    
    lifetime_kills = lifetime_filtered["Kills"].sum()
    lifetime_assists = lifetime_filtered["Assists"].sum() * (1/3)
    lifetime_deaths = lifetime_filtered["Deaths"].sum()
    
    if season_deaths == 0:
        season_level_kd = season_kills
        season_level_kda = season_kills + season_assists
    else:
        season_level_kd = season_kills/season_deaths
        season_level_kda = (season_kills + season_assists)/season_deaths
        
    if lifetime_deaths == 0:
        lifetime_level_kd = lifetime_kills
        lifetime_level_kda = lifetime_kills + lifetime_assists
    else:
        lifetime_level_kd = lifetime_kills/lifetime_deaths
        lifetime_level_kda = (lifetime_kills + lifetime_assists)/lifetime_deaths
        
    return season_level_kd, season_level_kda, lifetime_level_kd, lifetime_level_kda
    
def kill_death_squad(squad_size):
    
    global season_squad_kd, season_squad_kda, lifetime_squad_kd, lifetime_squad_kda
    
    season_filtered = season[season["Squad Size"] == squad_size]
    lifetime_filtered = lifetime[lifetime["Squad Size"] == squad_size]
    
    season_kills = season_filtered["Kills"].sum()
    season_assists = season_filtered["Assists"].sum() * (1/3)
    season_deaths = season_filtered["Deaths"].sum()
    
    lifetime_kills = lifetime_filtered["Kills"].sum()
    lifetime_assists = lifetime_filtered["Assists"].sum() * (1/3)
    lifetime_deaths = lifetime_filtered["Deaths"].sum()
    
    if season_deaths == 0:
        season_squad_kd = season_kills
        season_squad_kda = season_kills + season_assists
    else:
        season_squad_kd = season_kills/season_deaths
        season_squad_kda = (season_kills + season_assists)/season_deaths
        
    if lifetime_deaths == 0:
        lifetime_squad_kd = lifetime_kills
        lifetime_squad_kda = lifetime_kills + lifetime_assists
    else:
        lifetime_squad_kd = lifetime_kills/lifetime_deaths
        lifetime_squad_kda = (lifetime_kills + lifetime_assists)/lifetime_deaths

    return season_squad_kd, season_squad_kda, lifetime_squad_kd, lifetime_squad_kda

In [12]:
def wl_comparator():
    
    global wl_levels_table, wl_squads_table
    
    wl_levels = ["Average W/L"]
    wl_ratios = [avg_season_wl]
    lifetime_wl_ratios = [avg_lifetime_wl]
    
    wl_squads = ["Average W/l"]
    wl_ratios_squad = [avg_season_wl]
    lifetime_wl_ratios_squad = [avg_lifetime_wl]
    
    for level in levels:
        win_loss_level(level)
        wl_levels.append(level)
        wl_ratios.append(season_level_wl)
        lifetime_wl_ratios.append(lifetime_level_wl)
    
    for size in sizes:
        win_loss_squad(size)
        wl_squads.append(size)
        wl_ratios_squad.append(season_squad_wl)
        lifetime_wl_ratios_squad.append(lifetime_squad_wl)
        
    wl_levels_table = pd.DataFrame({"Map": wl_levels, "Season W/L": wl_ratios, "Lifetime W/L": lifetime_wl_ratios})
    wl_squads_table = pd.DataFrame({"Squad Size": wl_squads, "Season W/L": wl_ratios_squad, "Lifetime W/L": lifetime_wl_ratios_squad})
    
    return wl_levels_table, wl_squads_table

def kd_comparator():
    
    global kd_levels_table, kd_squads_table
    
    kd_levels = ["Average K/D"]
    kd_ratios = [avg_season_kd]
    kda_ratios = [avg_season_kda]
    lifetime_kd_ratios = [avg_lifetime_kd]
    lifetime_kda_ratios = [avg_lifetime_kda]
    
    kd_squads = ["Average K/D"]
    kd_ratios_squad = [avg_season_kd]
    kda_ratios_squad = [avg_season_kda]
    lifetime_kd_ratios_squad = [avg_lifetime_kd]
    lifetime_kda_ratios_squad = [avg_lifetime_kda]
    
    for level in levels:
        kill_death_level(level)
        kd_levels.append(level)
        kd_ratios.append(season_level_kd)
        kda_ratios.append(season_level_kda)
        lifetime_kd_ratios.append(lifetime_level_kd)
        lifetime_kda_ratios.append(lifetime_level_kda)
        
    for size in sizes:
        kill_death_squad(size)
        kd_squads.append(size)
        kd_ratios_squad.append(season_squad_kd)
        kda_ratios_squad.append(season_squad_kda)
        lifetime_kd_ratios_squad.append(lifetime_squad_kd)
        lifetime_kda_ratios_squad.append(lifetime_squad_kda)
        
    kd_levels_table = pd.DataFrame({"Map": kd_levels,
                                    "Season K/D": kd_ratios,
                                    "Season KA/D": kda_ratios,
                                    "Lifetime K/D": lifetime_kd_ratios,
                                    "Lifetime KA/D": lifetime_kda_ratios})
    kd_squads_table = pd.DataFrame({"Squad Size": kd_squads,
                                    "Season K/D": kd_ratios_squad,
                                    "Season KA/D": kda_ratios_squad,
                                    "Lifetime K/D": lifetime_kd_ratios_squad,
                                    "Lifetime KA/D": lifetime_kda_ratios_squad})
    
    return kd_levels_table, kd_squads_table

In [13]:
def get_scores_levels(level_name):
    
    global season_avg_score, lifetime_avg_score
    
    season_filtered = season_scores[season_scores["Map"] == level_name]
    lifetime_filtered = lifetime_scores[lifetime_scores["Map"] == level_name]
    season_count = len(season_filtered)
    lifetime_count = len(lifetime_filtered)
    
    season_total_score = season_filtered["Adjusted Score"].sum()
    lifetime_total_score = lifetime_filtered["Adjusted Score"].sum()
    
    if season_count == 0:
        season_avg_score = 0
    else:
        season_avg_score = season_total_score/season_count
    
    if lifetime_count == 0:
        lifetime_avg_score = 0
    else:
        lifetime_avg_score = lifetime_total_score/lifetime_count
    
    return level_name, season_avg_score, lifetime_avg_score

def get_scores_squads(squad_size):
    
    global season_avg_score, lifetime_avg_score
    
    season_scores_filtered = season_scores[season_scores["Squad Size"] == squad_size]
    lifetime_scores_filtered = lifetime_scores[lifetime_scores["Squad Size"] == squad_size]
    season_count = len(season_scores_filtered)
    lifetime_count = len(lifetime_scores_filtered)
    
    season_total_score = season_scores_filtered["Adjusted Score"].sum()
    lifetime_total_score = lifetime_scores_filtered["Adjusted Score"].sum()
    
    if season_count == 0:
        season_avg_score = 0
    else:
        season_avg_score = season_total_score/season_count
    
    if lifetime_count == 0:
        lifetime_avg_score = 0
    else:
        lifetime_avg_score = lifetime_total_score/season_count
    
    return squad_size, season_avg_score, lifetime_avg_score

In [14]:
def score_comparator():
    
    global score_levels_table, score_squads_table
    
    score_levels = ["Average Score"]
    season_level_scores = [avg_season_score]
    lifetime_level_scores = [avg_lifetime_score]
    
    score_squads = ["Average Score"]
    season_squad_scores = [avg_season_score]
    lifetime_squad_scores = [avg_lifetime_score]
    
    for level in levels:
        get_scores_levels(level)
        score_levels.append(level)
        season_level_scores.append(season_avg_score)
        lifetime_level_scores.append(lifetime_avg_score)
        
    for size in sizes:
        get_scores_squads(size)
        score_squads.append(size)
        season_squad_scores.append(season_avg_score)
        lifetime_squad_scores.append(lifetime_avg_score)
        
    score_levels_table = pd.DataFrame({"Map": score_levels,
                                       "Season Score": season_level_scores,
                                       "Lifetime Score": lifetime_level_scores})
    score_squads_table = pd.DataFrame({"Squad Size": score_squads,
                                       "Season Score": season_squad_scores,
                                       "Lifetime Score": lifetime_squad_scores})
    
    return score_levels_table, score_squads_table

In [15]:
def individual_performance_calculator(level_name):
    
    global josh_avg, asa_avg, ed_avg, luke_avg, josh_count, asa_count, ed_count, luke_count


    josh_filtered = josh_scores[josh_scores["Map"] == level_name]
    asa_filtered = asa_scores[asa_scores["Map"] == level_name]
    ed_filtered = ed_scores[ed_scores["Map"] == level_name]
    luke_filtered = luke_scores[luke_scores["Map"] == level_name]
    
    josh_count = len(josh_filtered)
    asa_count = len(asa_filtered)
    ed_count = len(ed_filtered)
    luke_count = len(luke_filtered)
    
    josh_total = josh_filtered["Adjusted Score"].sum()
    asa_total = asa_filtered["Adjusted Score"].sum()
    ed_total = ed_filtered["Adjusted Score"].sum()
    luke_total = luke_filtered["Adjusted Score"].sum()
    
    if josh_count == 0:
        josh_avg = 0
    else:
        josh_avg = josh_total/josh_count

    if asa_count == 0:
        asa_avg = 0
    else:
        asa_avg = asa_total/asa_count
        
    if ed_count == 0:
        ed_avg = 0
    else:
        ed_avg = ed_total/ed_count
        
    if luke_count == 0:
        luke_avg = 0
    else:
        luke_avg = luke_total/luke_count
    
    return level_name, josh_avg, asa_avg, ed_avg, luke_avg, josh_count, asa_count, ed_count, luke_count

In [16]:
def individual_perforamnce_constructor():
    
    global ip_table
    
    ip_score_levels = ["Average Score"]
    ip_josh_scores = [josh_all_levels_avg]
    ip_josh_games = [josh_count_season]
    ip_asa_scores = [asa_all_levels_avg]
    ip_asa_games = [asa_count_season]
    ip_ed_scores = [ed_all_levels_avg]
    ip_ed_games = [ed_count_season]
    ip_luke_scores = [luke_all_levels_avg]
    ip_luke_games = [luke_count_season]
    
    for level in levels:
        individual_performance_calculator(level)
        ip_score_levels.append(level)
        ip_josh_scores.append(josh_avg)
        ip_josh_games.append(josh_count)
        ip_asa_scores.append(asa_avg)
        ip_asa_games.append(asa_count)
        ip_ed_scores.append(ed_avg)
        ip_ed_games.append(ed_count)
        ip_luke_scores.append(luke_avg)
        ip_luke_games.append(luke_count)
        
    ip_table = pd.DataFrame({"Map": ip_score_levels,
                             "Josh Score": ip_josh_scores,
                             "Josh Games": ip_josh_games,
                             "Asa Score": ip_asa_scores,
                             "Asa Games": ip_asa_games,
                             "Ed Score": ip_ed_scores,
                             "Ed Games": ip_ed_games,
                             "Luke Score": ip_luke_scores,
                             "Luke Games": ip_luke_games})
    
    return ip_table

In [17]:
# def josh_performance(level_name):
    
#     global josh_avg, p1_avg, p2_avg, p3_avg, p4_avg, josh_count, p1_count, p2_count, p3_count, p4_count

#     josh_filtered = josh_scores[josh_scores["Map"] == level_name]
#     p1_filtered = p1_scores[p1_scores["Map"] == level_name]
#     p2_filtered = p2_scores[p2_scores["Map"] == level_name]
#     p3_filtered = p3_scores[p3_scores["Map"] == level_name]
#     p4_filtered = p4_scores[p4_scores["Map"] == level_name]
    
#     josh_count = len(josh_filtered)
#     p1_count = len(p1_filtered)
#     p2_count = len(p2_filtered)
#     p3_count = len(p3_filtered)
#     p4_count = len(p4_filtered)
    
#     josh_total = josh_filtered["Adjusted Score"].sum()
#     p1_total = p1_filtered["Adjusted Score"].sum()
#     p2_total = p2_filtered["Adjusted Score"].sum()
#     p3_total = p3_filtered["Adjusted Score"].sum()
#     p4_total = p4_filtered["Adjusted Score"].sum()
    
#     if josh_count == 0:
#         josh_avg = 0
#     else:
#         josh_avg = josh_total/josh_count

#     if p1_count == 0:
#         p1_avg = 0
#     else:
#         p1_avg = p1_total/p1_count
        
#     if p2_count == 0:
#         p2_avg = 0
#     else:
#         p2_avg = p2_total/p2_count
        
#     if p3_count == 0:
#         p3_avg = 0
#     else:
#         p3_avg = p3_total/p3_count

#     if p4_count == 0:
#         p4_avg = 0
#     else:
#         p4_avg = p4_total/p4_count
    
#     return level_name, josh_avg, p1_avg, p2_avg, p3_avg, p4_avg, josh_count, p1_count, p2_count, p3_count, p4_count

# def josh_performance_comparator():
    
#     global jp_table
    
#     jp_score_levels = ["Average Score"]
#     jp_josh_scores = [josh_all_levels_avg]
#     jp_p1_scores = [p1_all_levels_avg]
#     jp_p2_scores = [p2_all_levels_avg]
#     jp_p3_scores = [p3_all_levels_avg]
#     jp_p4_scores = [p4_all_levels_avg]
    
#     for level in levels:
#         josh_performance(level)
#         jp_score_levels.append(level)
#         jp_josh_scores.append(josh_avg)
#         jp_p1_scores.append(p1_avg)
#         jp_p2_scores.append([p2_avg])
#         jp_p3_scores.append(p3_avg)
#         jp_p4_scores.append(p4_avg)


#     jp_table = pd.DataFrame({"Map": jp_score_levels,
#                              "Josh Score": jp_josh_scores,
#                              "P1 Score": jp_p1_scores,
#                              "P2 Score": jp_p2_scores,
#                              "P3 Score": jp_p3_scores,
#                              "P4 Score": jp_p4_scores})
    
#     return jp_table

In [18]:

wl_comparator()
wl_levels_table.to_excel("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\win_loss_maps.xlsx")
wl_squads_table.to_excel("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\win_loss_squads.xlsx")

kd_comparator()
kd_levels_table.to_excel("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\kill_death_maps.xlsx")
kd_squads_table.to_excel("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\kill_death_squads.xlsx")

score_comparator()
score_levels_table.to_excel("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\score_maps.xlsx")
score_squads_table.to_excel("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\score_squads.xlsx")

individual_perforamnce_constructor()
ip_table.to_excel("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\individual_performance.xlsx")

# josh_performance_comparator()
# jp_table.to_excel("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\performance_comparator.xlsx")

In [20]:
# building a new table of variables I want for logistic regression

all_data_table = pd.DataFrame()

all_data_table["Warm-Up"] = season["Warm-Up"]
all_data_table["Map"] = season["Map"]
all_data_table["DEF Wins"] = season["DEF Wins"]
all_data_table["DEF Losses"] = season["DEF Losses"]
all_data_table["ATK Wins"] = season["ATK Wins"]
all_data_table["ATK Losses"] = season["ATK Losses"]
all_data_table["Outcome"] = season["Outcome"]
all_data_table["Score"] = josh_scores["Adjusted Score"]
all_data_table["Kills"] = season["Kills"]
all_data_table["Deaths"] = season["Deaths"]
all_data_table["Assists"] = season["Assists"]

all_data_table = all_data_table.dropna()

all_data_table.to_csv("C:\\Users\\aethe\\OneDrive\\Desktop\\stats\\data for r.csv")

In [21]:
'''
IDEA

Can maybe use replace function or can use score adjuster
method if need be, but idea is to replace win values
with 1 and loss values with 0 for data to R.
Goal is to use linear model with clear predicted outcome
in lieu of difficult-to-interpret logistic regression.
'''

'\nIDEA\n'